In [2]:
# Bibliotecas genéricas

import pandas as pd 
import numpy as np
import requests
import time
import json
import os   
import sys
import re
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
from matplotlib.ticker import MaxNLocator
from matplotlib import rcParams
from datetime import timedelta
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import concurrent
import plotly.express as px


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Dados de Voos

In [3]:
destino = np.array(['SBKP', 'SBGR', 'SBBR', 'SBCF', 'SBSP', 'SBRJ', 'SBFL', 'SBCT',
       'SBRF', 'SBPA', 'SBSV', 'SBGL'], dtype=object)

#### Dados da ANAC

In [6]:
# Dados de voos de vra 

# SBGR - Guarulhos

# Listar todos os arquivos na pasta 'vra'
vra_files = [os.path.join('vra', f) for f in os.listdir('vra') if f.endswith('.csv')]

# Concatenar todos os arquivos da pasta 'vra' que atendem à condição
vra_list = []
for file in vra_files:
    df = pd.read_csv(file, sep=';')
    df_filtered = df[df['Sigla ICAO Aeroporto Destino'].isin(destino)]
    vra_list.append(df_filtered)

vra = pd.concat(vra_list, ignore_index=True)

# Converter colunas de datas do vra para o padrão internacional ISO 8601 com horário

vra['Partida Real'] = pd.to_datetime(vra['Partida Real'],  format='%d/%m/%Y %H:%M')
vra['Chegada Real'] = pd.to_datetime(vra['Chegada Real'], format='%d/%m/%Y %H:%M')
vra['Partida Prevista'] = pd.to_datetime(vra['Partida Prevista'],  format='%d/%m/%Y %H:%M')
vra['Chegada Prevista'] = pd.to_datetime(vra['Chegada Prevista'],  format='%d/%m/%Y %H:%M')
vra['Horário de Referência para Partida'] = np.where(vra['Partida Real'].isnull(), vra['Partida Prevista'], vra['Partida Real'])
vra['Horário de Referência para Chegada'] = np.where(vra['Chegada Real'].isnull(), vra['Chegada Prevista'], vra['Chegada Real'])

vra.head()

/tmp/ipykernel_31261/2028365739.py:11: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=';')
/tmp/ipykernel_31261/2028365739.py:11: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=';')
/tmp/ipykernel_31261/2028365739.py:11: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=';')
/tmp/ipykernel_31261/2028365739.py:11: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=';')
/tmp/ipykernel_31261/2028365739.py:11: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=';')


,Sigla ICAO Empresa Aérea,Empresa Aérea,Número Voo,Código DI,Código Tipo Linha,Modelo Equipamento,Número de Assentos,Sigla ICAO Aeroporto Origem,Descrição Aeroporto Origem,Partida Prevista,...,Descrição Aeroporto Destino,Chegada Prevista,Chegada Real,Situação Voo,Justificativa,Referência,Situação Partida,Situação Chegada,Horário de Referência para Partida,Horário de Referência para Chegada
0,ACN,AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA),5945,6,N,C208,0,SBGL,AEROPORTO INTERNACIONAL DO RIO DE JANEIRO (GAL...,NaT,...,CONGONHAS - SÃO PAULO - SP - BRASIL,NaT,2022-11-01 16:46:00,REALIZADO,NaN,01/11/2022 00:00:00,NaN,NaN,2022-11-01 14:48:00,2022-11-01 16:46:00
1,ACN,AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA),5122,4,N,C208,9,SBGL,AEROPORTO INTERNACIONAL DO RIO DE JANEIRO (GAL...,NaT,...,CONGONHAS - SÃO PAULO - SP - BRASIL,NaT,2022-11-02 11:58:00,REALIZADO,NaN,02/11/2022 00:00:00,NaN,NaN,2022-11-02 09:45:00,2022-11-02 11:58:00
2,ACN,AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA),5950,2,N,C208,9,SBGL,AEROPORTO INTERNACIONAL DO RIO DE JANEIRO (GAL...,NaT,...,CONGONHAS - SÃO PAULO - SP - BRASIL,NaT,2022-11-02 14:19:00,REALIZADO,NaN,02/11/2022 00:00:00,NaN,NaN,2022-11-02 12:24:00,2022-11-02 14:19:00
3,ACN,AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA),5960,2,N,C208,0,SBGL,AEROPORTO INTERNACIONAL DO RIO DE JANEIRO (GAL...,NaT,...,CONGONHAS - SÃO PAULO - SP - BRASIL,NaT,2022-12-20 11:27:00,REALIZADO,NaN,20/12/2022 00:00:00,NaN,NaN,2022-12-20 09:34:00,2022-12-20 11:27:00
4,ACN,AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA),5927,2,N,C208,0,SBGL,AEROPORTO INTERNACIONAL DO RIO DE JANEIRO (GAL...,NaT,...,CONGONHAS - SÃO PAULO - SP - BRASIL,NaT,2023-02-20 10:40:00,REALIZADO,NaN,20/02/2023 00:00:00,NaN,NaN,2023-02-20 08:43:00,2023-02-20 10:40:00


##### Dados de recomendacao de seguranca

In [7]:
# Carregando dados de ocorrênncia

recomendacao = pd.read_csv('RECOMENDACAO_SEGURANCA.csv', sep=';')
recomendacao.head()

Atualizado em: 2024-10-07
NUMERO_OCORRENCIA NUMERO_PROCESSO      CATEGORIA_OCORRENCIA MATRICULA DATA_OCORRENCIA      NRO_RECOM_SEGURANCA TXT_RECOM_SEGURANCA                                CLASS_RECOM_SEGURANCA DATA_RECEBIMENTO_RECOM AREA_COMPETENTE STATUS_ANAC_RECOMENDACAO PRAZO_RESPOSTA_RECOM_SEGURANCA TEXTO_RESPOSTA                                                 DATA_RESPOSTA
26/2011           00065.136888/2012-13 Acidente             PROMO     2011-06-17 00:00:00. RSV (A) 382/2012    Reavaliar as condições operacionais do Helipont... Outros                2012-09-05 00:00:00.   SIA             Não Aplicável            2013-01-03 00:00:00.           Recebido o Memorando nº 55/2015/GCPO/SIA, de 15...      2014-06-11 00:00:00.
                  00065.136890/2012-84 Acidente             PROMO     2011-06-17 00:00:00. RSV (A) 381/2012    Atuar junto aos operadores de helicóptero para ... Outros                2012-09-05 00:00:00.   OUTRA           Cumprida                 2013-01-03 00:00:00.           NaN                                                     2014-06-11 00:00:00.
                  00065.138161/2012    Acidente             PROMO     2011-06-17 00:00:00. RSV (A) 378/2012    Incrementar a fiscalização das aeronaves utiliz... Fiscalização          2012-09-05 00:00:00.   OUTRA           Cumprida                 2013-01-03 00:00:00.           A RSV está cumprida, considerando que o Sistema...      2015-09-22 00:00:00.
                                                                                           RSV (A) 377/2012    Incrementar a fiscalização das aeronaves em ope... Fiscalização          2012-09-05 00:00:00.   OUTRA           Cumprida                 2013-01-03 00:00:00.           A RSV está cumprida, considerando que o Sistema...      2015-12-21 00:00:00.

#### Dados de ocorrências 

In [5]:
# Carregando dados de ocorrênncia

ocorrencia = pd.read_csv('V_OCORRENCIA_AMPLA.csv', sep=';')
ocorrencia.head()

,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,Atualizado em: 2024-10-07
Numero_da_Ocorrencia,Numero_da_Ficha,Operador_Padronizado,Classificacao_da_Ocorrencia,Data_da_Ocorrencia,Hora_da_Ocorrencia,Municipio,UF,Regiao,Descricao_do_Tipo,ICAO,Latitude,Longitude,Tipo_de_Aerodromo,Historico,Matricula,Categoria_da_Aeronave,Operador,Tipo_de_Ocorrencia,Fase_da_Operacao,Operacao,Danos_a_Aeronave,Aerodromo_de_Destino,Aerodromo_de_Origem,Lesoes_Fatais_Tripulantes,Lesoes_Fatais_Passageiros,Lesoes_Fatais_Terceiros,Lesoes_Graves_Tripulantes,Lesoes_Graves_Passageiros,Lesoes_Graves_Terceiros,Lesoes_Leves_Tripulantes,Lesoes_Leves_Passageiros,Lesoes_Leves_Terceiros,Ilesos_Tripulantes,Ilesos_Passageiros,Lesoes_Desconhecidas_Tripulantes,Lesoes_Desconhecidas_Passageiros,Lesoes_Desconhecidas_Terceiros,Modelo,CLS,Tipo_ICAO,PMD,Numero_de_Assentos,Nome_do_Fabricante,PSSO
25626,202096550,MINIST.JUST.SECRET.NAC.DE SEG.PUB.,Acidente,2020-10-08,16:00,CORUMBÁ,MS,Centro-Oeste,FALHA OU MAU FUNCIONAMENTO DO MOTOR,Fora de Aeródromo,"-17,4417","-56,8269",-,"A aeronave decolou do Aeroporto Internacional de Corumbá (SBCR), Corumbá, MS, com destino ao Aeródromo/Hotel Porto Jofre (SJQI), MT, por volta das 15h00min UTC, a fim de retornar à base operacional, com dois pilotos e um graduado operacional (fiel) a bordo. A aproximadamente 5 minutos para o pouso, o alarme de perda de rotação soou, ocorrendo, na sequência, uma perda de potência no motor da aeronave. O piloto prosseguiu para um pouso de emergência em uma área descampada à sua frente. Após o toque, houve a quebra dos esquis da aeronave e seu tombamento lateral. Os pilotos sofreram lesões graves e o terceiro tripulante sofreu lesões leves. A aeronave teve danos substanciais.",PRMJZ,ADF,MINIST.JUST.SECRET.NAC.DE SEG.PUB.,SCF-PP,Cruzeiro,Operação Pública,Substancial,SJQI,SBCR,NaN,NaN,NaN,2,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AS 350 B2,H1T,AS50,2250,6,EUROCOPTER FRANCE,verdadeiro
26304,202056314,AERO AGRICOLA ROSARIENSE LTDA,Acidente,2020-12-09,21:30:00,SANT'ANA DO LIVRAMENTO,RS,Sul,COMBUSTÍVEL,Fora de Aeródromo,"-30,6056","-55,0564",-,"A aeronave decolou da pista pouso eventual da Fazenda Colorado, município de Santana do Livramento, RS, às 18h10min (UTC), a fim de realizar pulverização de defensivos agrícolas em lavoura de arroz, com um piloto a bordo. Após 20 minutos de voo, a aeronave teve um apagamento do motor. O piloto realizou pouso de emergência em lavoura de soja, porém antes do pouso colidiu o trem de pouso principal em uma valeta. A aeronave teve danos substanciais. O tripulante saiu ileso.",PRREN,S11,AERO AGRICOLA ROSARIENSE LTDA,FUEL,Voo a baixa altura,Operação Agrícola,Substancial,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,A188B,L1P,C188,1905,1,CESSNA AIRCRAFT,verdadeiro
25763,202041291,KAIO GENARIO FERREIRA DE MELO,Acidente,2020-10-15,11:00,RIO CLARO,RJ,Sudeste,PERDA DE CONTROLE EM VOO,Fora de Aeródromo,"-22,7031","-43,9947",-,"A aeronave decolou de local desconhecido para destino desconhecido com um piloto e um passageiro a bordo. Durante o voo, a aeronave colidiu contra o terreno no alto de um morro no distrito de Passa Três, Rio Claro (RJ). A aeronave teve danos substanciais. O piloto e o passageiro sofreram lesões fatais.",PRESK,TPP,KAIO GENARIO FERREIRA DE MELO,LOC-I,Cruzeiro,Voo Privado,Substancial,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R44,H1P,R44,1088,4,ROBINSON HELICOPTER,verdadeiro
26123,202060427,MARCIO ROBERTO SALES DE ARAUJO,Acidente,2020-11-19,15:00,IPIXUNA DO PARÁ,PA,Norte,OUTROS,Aeródromo não cadastrado,"-2,52357","-47,5181",NaN,"A AERONAVE DECOLOU DE UMA PISTA DESCONHECIDA COM DESTINO A UMA PISTA NÃO REGISTRADA LOCALIZADA NO MUNICÍPIO DE IPIXUNA DO PARÁ, PA, EM HORÁRIO DESCONHECIDO, COM DOIS OCUPANTES A BORDO E CARGA DE ENTORPECENTES.DURANTE O POUSO NA PISTA NÃO REGISTRADA A AERONAVE FOI INTERCEPTADA POR AGENTES DA POLÍCIA FEDERAL. A AERONAVE SAIU LATERALMENTE PELA DIREITA E PAROU FORA DOS LIMITES DA PISTA, APÓS SER ALVEJADA POR TIROS. A AERO

### Dados de esperas



In [7]:
# Carregando dados de esperas em voo

esperas = pd.read_csv('esperas.csv')

esperas['hora'] = pd.to_datetime(esperas['hora'], unit='ms')

esperas[esperas['esperas'] > 0 ].head(20)

,esperas,hora,aero
49,1,2022-06-01 01:00:00,SBCT
142,2,2022-06-01 22:00:00,SBGR
153,6,2022-06-01 09:00:00,SBKP
154,3,2022-06-01 10:00:00,SBKP
254,1,2022-06-01 14:00:00,SBSP
260,1,2022-06-01 20:00:00,SBSP
263,1,2022-06-01 23:00:00,SBSP
374,2,2022-06-02 14:00:00,SBCF
385,1,2022-06-01 01:00:00,SBCT
550,2,2022-06-01 22:00:00,SBGR


### Dados Metereológicos
Link para interpretar dados metereológicos: [[link]](https://ajuda.decea.mil.br/base-de-conhecimento/como-decodificar-o-metar-e-o-speci/)

Exemplo: 'METAR SBGL 131000Z 31015G27KT 280V350 4000 1800N R10/P2000 +TSRA FEW005 FEW010CB SCT018 BKN025 10/03 Q0995 REDZ WS R10 W12/H75='
Também é possível uitlizar bibliotecas para interpretar os dados metereológicos, como a `metar` [[link]](https://pypi.org/project/metar/)


Instalando biblioteca para interpretar dados metar: 


In [8]:
!pip install metar

from metar import Metar

Defaulting to user installation because normal site-packages is not writeable


Função para transformar dados


In [9]:
def get_metar_data(x):
    try:
        metar = Metar.Metar(x)
        # Organizar os dados em um dicionário
        data = {
            'station': metar.station_id,
            'time': metar.time,
            'wind_direction': metar.wind_dir.value() if metar.wind_dir else None,
            'wind_speed': metar.wind_speed.value('KT') if metar.wind_speed else None,
            'temperature': metar.temp.value('C') if metar.temp else None,
            'dew_point': metar.dewpt.value('C') if metar.dewpt else None,
            'visibility': metar.vis.value('M') if metar.vis else None,
            'pressure': metar.press.value('MB') if metar.press else None,
            'sky_conditions': metar.sky_conditions() if metar.sky_conditions() else None
        }
        return data # Tenta dividir 10 por y
    except:
        return ''

#### METAF


In [10]:
# Carregando os dados de METAF 


if not os.path.exists('metaf_translated.csv'):
    metaf = pd.read_csv('metaf.csv')

    metaf['hora'] = pd.to_datetime(metaf['hora'], unit='ms')

    metaf['metaf'] = metaf['metaf'].str.replace('METAF', 'METAR')

    # Traduzindo metaf

    metaf['metar_translated'] = metaf['metaf'].apply(lambda x: get_metar_data(x))
    data = metaf['metar_translated']
    new_columns = pd.DataFrame(data.to_dict()).T
    metaf = pd.concat([metaf, new_columns], axis=1)
    metaf.to_csv('metaf_translated.csv', index=False)   
else:
    metaf = pd.read_csv('metaf_translated.csv')
    
metaf.head()

metaf['hora'] = pd.to_datetime(metaf['hora'])



#### METAR


In [11]:

# Verifica se já existe o arquivo metar_translated.csv

if not os.path.exists('metar_translated.csv'):

    # Carregando dados do METAR

    metar = pd.read_csv('metar.csv')

    # Transformando colunas de data

    metar['hora'] = pd.to_datetime(metar['hora'], unit='ms')

    # Traduzindo metar 

    metar['metar_translated'] = metar['metar'].apply(lambda x: get_metar_data(x))
    data = metar['metar_translated']
    new_columns = pd.DataFrame(data.to_dict()).T
    metar = pd.concat([metar, new_columns], axis=1)
    metar.to_csv('metar_translated.csv', index=False)   
else:
    metar = pd.read_csv('metar_translated.csv')
    
metar.head()
    
metar['hora'] = pd.to_datetime(metar['hora'])
    
    


### Juntando Informações

Inserindo informações da ANAC com os parâmetros das condições dos aeroportos

In [12]:
# Check if the file already exists
if not os.path.exists('vra_full.csv'):
    # Perform the merging operations
    vra_full = pd.merge_asof(
        vra.sort_values('Horário de Referência para Partida', ascending=True),
        metar.rename(columns={'aero': 'Sigla ICAO Aeroporto Origem'}).sort_values('hora', ascending=True),
        left_on='Horário de Referência para Partida',
        right_on='hora',
        by='Sigla ICAO Aeroporto Origem',
        direction='nearest'
    )

    # Change suffix new columns names in vra on merged_df to previsto
    # example temperature -> temperature_previsto
    vra_full.columns = [f'{col}_orig' if col in metar.columns else col for col in vra_full.columns]

    vra_full = pd.merge_asof(
        vra_full.sort_values('Horário de Referência para Chegada', ascending=True),
        metar.rename(columns={'aero': 'Sigla ICAO Aeroporto Destino'}).sort_values('hora', ascending=True),
        left_on='Horário de Referência para Chegada',
        right_on='hora',
        by='Sigla ICAO Aeroporto Destino',
        direction='nearest'
    )

    vra_full.columns = [f'{col}_dest' if col in metar.columns else col for col in vra_full.columns]

    # Drop columns hora, time_prev and time_dest
    vra_full.drop(columns=['time_orig', 'time_dest'], inplace=True)

    vra_full = pd.merge_asof(
        vra_full.sort_values('Horário de Referência para Partida', ascending=True),
        esperas.rename(columns={'aero': 'Sigla ICAO Aeroporto Origem'}).sort_values('hora', ascending=True),
        left_on='Horário de Referência para Partida',
        right_on='hora',
        by='Sigla ICAO Aeroporto Origem',
        direction='nearest'
    )

    # Change hora column name to hora_orig
    vra_full.columns = [f'{col}_orig' if col in esperas.columns else col for col in vra_full.columns]

    vra_full = pd.merge_asof(
        vra_full.sort_values('Horário de Referência para Chegada', ascending=True),
        esperas.rename(columns={'aero': 'Sigla ICAO Aeroporto Destino'}).sort_values('hora', ascending=True),
        left_on='Horário de Referência para Chegada',
        right_on='hora',
        by='Sigla ICAO Aeroporto Destino',
        direction='nearest'
    )

    # Change hora column name to hora_dest
    vra_full.columns = [f'{col}_dest' if col in esperas.columns else col for col in vra_full.columns]

    # Save the merged dataframe to a CSV file
    vra_full.to_csv('vra_full.csv', index=False)
else:
    # Load the dataframe from the CSV file
    vra_full = pd.read_csv('vra_full.csv')

vra_full.head()

,Sigla ICAO Empresa Aérea,Empresa Aérea,Número Voo,Código DI,Código Tipo Linha,Modelo Equipamento,Número de Assentos,Sigla ICAO Aeroporto Origem,Descrição Aeroporto Origem,Partida Prevista,...,wind_speed_dest,temperature_dest,dew_point_dest,visibility_dest,pressure_dest,sky_conditions_dest,esperas_orig,hora_orig.1,esperas_dest,hora_dest.1
0,GLO,GOL LINHAS AÉREAS S.A. (EX- VRG LINHAS AÉREAS ...,2028,2,N,B38M,186,SBGL,AEROPORTO INTERNACIONAL DO RIO DE JANEIRO (GAL...,NaN,...,2.0,25.0,24.0,10000.0,1012.0,a few clouds at 1300 feet; broken clouds at 19...,0,2022-06-01 00:00:00,0,2022-06-01 00:00:00
1,SID,SIDERAL LINHAS AÉREAS LTDA.,9412,0,C,B73Y,0,SBCF,TANCREDO NEVES - CONFINS - MG - BRASIL,2022-06-01 00:40:00,...,3.0,16.0,15.0,10000.0,1019.0,scattered clouds at 800 feet; overcast at 1700...,0,2022-06-01 01:00:00,0,2022-06-01 02:00:00
2,SID,SIDERAL LINHAS AÉREAS LTDA.,9306,7,N,B73Y,0,SBBR,PRESIDENTE JUSCELINO KUBITSCHEK - BRASÍLIA - D...,2022-06-01 00:15:00,...,3.0,16.0,15.0,10000.0,1019.0,scattered clouds at 800 feet; overcast at 1700...,0,2022-06-01 00:00:00,0,2022-06-01 02:00:00
3,TTL,TOTAL LINHAS AÉREAS S.A.,5683,0,C,B722,0,SBGR,GUARULHOS - GOVERNADOR ANDRÉ FRANCO MONTORO - ...,2022-06-01 02:00:00,...,2.0,22.0,20.0,10000.0,1015.0,scattered clouds at 1800 feet; overcast at 300...,0,2022-06-01 02:00:00,0,2022-06-01 03:00:00
4,SID,SIDERAL LINHAS AÉREAS LTDA.,9303,0,C,B734,0,SBGR,GUARULHOS - GOVERNADOR ANDRÉ FRANCO MONTORO - ...,2022-06-01 02:40:00,...,2.0,22.0,20.0,10000.0,1015.0,scattered clouds at 1800 feet; overcast at 300...,0,2022-06-01 03:00:00,0,2022-06-01 04:00:00


Jutando informações de esperas em cabeceiras